In [ ]:
%matplotlib inline


# Find Photodiode Events
In this example, we use ``pd-parser`` to find photodiode events and
align them to behavior. Then, we save the data to BIDS format.


In [ ]:
# Authors: Alex Rockhill <aprockhill@mailbox.org>
#
# License: BSD (3-clause)

Simulate data and use it to make a raw object

We'll make an `mne.io.Raw object` so that we can save out some random
data with a photodiode event channel in it in `fif` format (a commonly used
electrophysiology data format).



In [ ]:
import os.path as op
import numpy as np

import mne
from mne.utils import _TempDir

import pd_parser
from pd_parser.parse_pd import _read_raw, _to_tsv

out_dir = _TempDir()
print(f'After running this example, you can find the data here: {out_dir}')

# simulate photodiode data
n_events = 300
prop_corrupted = 0.01
raw, beh, events, corrupted_indices = \
    pd_parser.simulate_pd_data(n_events=n_events,
                               prop_corrupted=prop_corrupted)

# make fake electrophysiology data
info = mne.create_info(['ch1', 'ch2', 'ch3'], raw.info['sfreq'],
                       ['seeg'] * 3)
raw2 = mne.io.RawArray(np.random.random((3, raw.times.size)) * 1e-6, info)
raw2.info['lowpass'] = raw.info['lowpass']  # these must match to combine
raw.add_channels([raw2])
# bids needs these data fields
raw.info['dig'] = None
raw.info['line_freq'] = 60

fname = op.join(out_dir, 'sub-1_task-mytask_raw.fif')
raw.save(fname)

# roundtrip so that raw is properly loaded from disk and has a filename
raw = _read_raw(fname)

Make behavior data

We'll make a dictionary with lists for the events that are time-stamped when
the photodiode was turned on and other events relative to those events.
We'll add some noise to the time-stamps so that we can see how behavior
might look in an experimental setting.
Let's make a task where there is a fixation stimulus, then a go cue,
and a then response as an example.



In [ ]:
np.random.seed(12)
# add some noise to make it harder to align, use just over
# the exclusion of 0.03 to make some events excluded
offsets = np.random.random(n_events) * 0.035 - 0.0125
# in this example, the fixation would always be 700 ms
# after which point a cue would appear which is the "go time"
go_time = np.repeat(0.7, n_events)
# let's make the response time between 0.5 and 1.5 seconds uniform random
response_time = list(go_time + np.random.random(n_events) + 1.5)
for i in [10, 129, 232, 288]:
    response_time[i] = 'n/a'  # make some no responses


# put in dictionary to be converted to tsv file
beh['fix_onset_time'] = beh['time'] + offsets
beh['go_time'] = go_time
beh['response_time'] = response_time
behf = op.join(out_dir, 'sub-1_task-mytask_beh.tsv')
# save behavior file out
_to_tsv(behf, beh)

Use the interactive graphical user interface (GUI) to find parameters

On the documentation webpage, this is example is not interactive,
but if you download it as a jupyter notebook and run it or copy the code
into a console running python (ipython recommended), you can see how to
interact with the photodiode data to pick reasonable parameters by
following the instructions.



In [ ]:
pd_parser.find_pd_params(raw, pd_ch_names=['pd'])

Find the photodiode events relative to the behavioral timing of interest

This function will use the default parameters or the parameters you
found from :func:`pd_parser.find_pd_parameters` to find and align the
photodiode events, excluding events that were off because the computer
hung up on computation, for instance. That data is saved in the same folder
as the raw file (in this case, a temperary directory generated by
:func:`_TempDir`). The data can be used directly, or it can be accessed via
:func:`pd_parser.pd_parser_save_to_bids` to store it in the brain imagine
data structure (BIDS) standardized format before using it.



In [ ]:
pd_parser.parse_pd(raw, beh=beh, pd_ch_names=['pd'], max_len=1.5)

Add events relative to the photodiode events

The photodiode is usually sychronized to one event (e.g. the fixation
so that if the deflections caused by the photodiode are large enough
to influence other channels through amplifier interactions it doesn't
cause issues with the analysis) so often the events of interest are
relative to the photodiode event. In the task a timer can be started at the
photodiode event and checked each time a subsequent event occurs.
These events should then be recorded in tsv file, which can be passed to
``pd-parser`` in order to add the events.
Note: if more than one photodiode event is used, the parser can be
used for each event separately using the keyword `add_event=True`.



In [ ]:
pd_parser.add_relative_events(
    raw, beh,
    relative_event_keys=['go_time', 'response_time'],
    relative_event_names=['Go Cue', 'Response'])

Save data to BIDS format

This saves our data to BIDS format so that it's ready to be analyzed in a
reproducible way; BIDS requires all the files the BIDS community has deemed
necessary for analysis, so you should have everything you need to continue
on with an analysis at this point. See https://bids.neuroimaging.io/ and
https://mne.tools/mne-bids/ for more information about BIDS.



In [ ]:
pd_parser.save_to_bids(op.join(out_dir, 'bids_dir'), fname,
                       sub='1', task='mytask')